#### Getting started with LCEL 

- LCEL stands from Langchain Expression Language 
- Here we will make a translator that will translate English to Other Language .
- way of getting started to LLMs making .

##### Here what we will do 

- Using language model 
- Using PromptTemplate and OutputParser 
- using the LCEL to chain the model 
- Debugging and trace the application on LangSmith 
- Deploying the application on Langserve 


In [1]:
!pip install langchain

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [3]:
!pip install langchain_groq

In [ ]:
# Using the open source Groq model Gemma2-9b-It

from langchain_groq import ChatGroq
model =ChatGroq(model="Gemma2-9b-It" , groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000027C06189FF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000027C07184BE0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
!pip install langchain_core

In [24]:
from langchain_core.messages  import HumanMessage , SystemMessage 

''' 
Why required ? 

- We must know which msg is sent by the human being , or which msg is sent as instruction .

'''

messages = [
    SystemMessage(content= "Translate the following from English to French ") ,
    HumanMessage(content= " Hello , How are you ? ") 
]

result = model.invoke(messages)  # save it 
result 

AIMessage(content='Bonjour, comment allez-vous ?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 24, 'total_tokens': 35, 'completion_time': 0.02, 'prompt_time': 0.002132036, 'queue_time': 0.237575529, 'total_time': 0.022132036}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ecbb5636-48bc-4f50-8364-6a503fa0cefb-0', usage_metadata={'input_tokens': 24, 'output_tokens': 11, 'total_tokens': 35})

In [15]:
# Since the result is very complex i need the only content part , so doing the Stringparser 

'''  
When you already have the LLM response and want to extract structured data -> parse
When you're working with LangChain pipelines and need end-to-end processing -> invoke 

'''

from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

parser.invoke(result)  # Take the content and show it 


'Bonjour, comment allez-vous ? \n'

In [16]:
# Now using LCEL we can chain the components 
chain = model|parser
chain.invoke(messages)   # Here it will take the message and pass it to model and then call the parser to parse it 



"Bonjour, comment allez-vous ? \n\n\nLet me know if you have any other phrases you'd like me to translate! 😊\n"

In [26]:
# Instead of passing the messages as list , we can pass it via PromptTemplate

from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate into following into {language}:"

" Earlier we are giving the prompt in a static way , now giving in a dynamic way "
prompt = ChatPromptTemplate(
    [
        ("system" , generic_template) , 
        ("user" , "{text}")
    ]
)



In [27]:
result = prompt.invoke(
    {"language":"French" , "text":"Hello"} 
)

# The output show , how the prompt is getting converted into list


In [28]:
result 

ChatPromptValue(messages=[SystemMessage(content='Translate into following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [29]:
result.to_messages()   # to print the message 

[SystemMessage(content='Translate into following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [22]:
# now chaining

chain = prompt|model|parser
chain.invoke(
    {"language":"French" , "text":"Hello"} 
)

'Bonjour \n'